<a href="https://colab.research.google.com/github/nspiegeln/iml_2022/blob/main/task4_main20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%reset -f


In [6]:
import pandas as pd
import torch
from torch import nn
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses, metrics
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import mean_squared_error


#load data
from google.colab import drive
drive.mount('/content/gdrive')
path = "gdrive/MyDrive/task4/"

df_train= pd.read_csv(path + "new_train_features2.csv")
df_train_labels= pd.read_csv(path + "train_labels.csv")
df_pretrain= pd.read_csv(path +"new_pretrain_features2.csv")
df_pretrain_labels= pd.read_csv(path +"pretrain_labels.csv")
df_test= pd.read_csv(path +"new_test_features2.csv")

# df_train_values = df_train.iloc[:,2::]
# df_pretrain_values = df_pretrain.iloc[:,2::]

# df_train_lab = df_train_labels.iloc[:,1]
# df_pretrain_lab = df_pretrain_labels.iloc[:,1]


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [10]:
train = df_train.values #dataset with known homo-lumo gap
pretrain = df_pretrain.values #dataset with known lumo value

train_labels = df_train_labels.iloc[:,1].values
pretrain_labels = df_pretrain_labels.iloc[:,1].values

x_pretrain, x_pretest, y_pretrain, y_pretest = train_test_split(pretrain, pretrain_labels, test_size = 0.1)
x_test = df_test.values
id_test = df_test.iloc[:,0]


In [ ]:
print(train)

In [3]:
# PRE-PROCESSING of features according to autoencoder
# pre-train set

#hyperparameters
dim = 200
epochs = 100         #resuls still decrease after epoch 100, @epoch=100 loss=3.4e-3
batch_size = 64

# create encoder and decoder to reduce dimensionality of features
class Autoencoder(Model):
  def __init__(self, dim):
    super(Autoencoder, self).__init__()
    self.dim = dim
    self.encoder = tf.keras.Sequential([
      # layers.Dense(1000, activation='relu'), 
      # layers.Dense(800, activation='relu'),
      layers.Dense(500, activation='relu'),
      layers.Dense(dim, activation='relu'),
    ])
    self.trainor = tf.keras.Sequential([
      layers.Dense(100, activation='relu'),
      
    ])
    self.predictor = tf.keras.Sequential([
      layers.Dense(100, activation='relu'),
      layers.Dense(50, activation='relu'),
      layers.Dense(1),
    ])                                     

  def call(self, x):
    encoded = self.encoder(x)
    trained = self.trainor(encoded)
    predicted = self.predictor(trained)
    return predicted

autoencoder = Autoencoder(dim)
sgd = SGD(learning_rate=0.1, decay=0.9, momentum=0.9) #changed decay from 1e-2 to 0.9 loss = 3.5e-3
autoencoder.compile(optimizer='sgd', loss=losses.MeanSquaredError())
autoencoder.fit(x_pretrain, y_pretrain, epochs=epochs, batch_size = batch_size, shuffle=True, validation_data=(x_pretest, y_pretest))

# y_predpretrain = autoencoder.predict(x_pretrain)
#%%


Epoch 1/100
704/704 [==============================] - 48s 67ms/step - loss: nan - val_loss: nan
Epoch 2/100
704/704 [==============================] - 43s 62ms/step - loss: nan - val_loss: nan
Epoch 3/100
704/704 [==============================] - 42s 59ms/step - loss: nan - val_loss: nan
Epoch 4/100
704/704 [==============================] - 41s 58ms/step - loss: nan - val_loss: nan
Epoch 5/100
396/704 [===============>..............] - ETA: 18s - loss: nan

KeyboardInterrupt: ignored

In [ ]:

# Transfer learning to HOMO-LUMO gap learning
# freeze everything but last layer
for layer in autoencoder.layers[:-1]:
  layer.trainable = False


dim = 1
epochs = 800   #loss decreases to loss=6.8e-3 after 1000 epochs
class AutoencoderNew(Model):
  def __init__(self, dim):
    super(AutoencoderNew, self).__init__()
    self.dim = dim
    self.encoder = tf.keras.Sequential([
      autoencoder,
      layers.Dense(50, activation='relu'),
      layers.Dense(100, activation='relu'),
      layers.Dense(200, activation='relu'),
      layers.Dense(200, activation='relu'),
      layers.Dense(200, activation='relu'),
      layers.Dense(500, activation='relu'),
      layers.Dense(500, activation='relu'),
      layers.Dense(200, activation='relu'),
      layers.Dense(200, activation='relu'),
      layers.Dense(100, activation='relu'),
      layers.Dense(50, activation='relu'),
      layers.Dense(1),
    ])                                    

  def call(self, x):
    encoded = self.encoder(x)
    return encoded

model = AutoencoderNew(dim)
sgd = SGD(learning_rate=0.1, decay=0.9, momentum=0.9)

model.compile(optimizer='sgd', loss=losses.MeanSquaredError())
model.fit(train, train_labels, epochs=epochs, shuffle=True)


#predict homolumo gap for testset
y_test = model.predict(x_test)


#export solution
df_evaltest = pd.DataFrame(y_test, columns = ['y'])
df_evaltest.insert(0, 'Id', id_test)
df_evaltest.to_csv('solutiontot20.csv', index=False, index_label=False, encoding = 'utf-8')